In [1]:
import pandas as pd
import numpy as np
import os
import importlib
import sys
import copy

In [2]:
backtesting_path = r'C:\Users\vinayak\Desktop\Backtesting'
print("path before", sys.path)
if backtesting_path not in sys.path:
    print("Backtesting was not in sys path")
    sys.path.append(backtesting_path)
print("path after", sys.path)
print()

path before ['c:\\Program Files\\Python312\\python312.zip', 'c:\\Program Files\\Python312\\DLLs', 'c:\\Program Files\\Python312\\Lib', 'c:\\Program Files\\Python312', '', 'C:\\Users\\vinayak\\AppData\\Roaming\\Python\\Python312\\site-packages', 'c:\\Program Files\\Python312\\Lib\\site-packages', 'c:\\Program Files\\Python312\\Lib\\site-packages\\win32', 'c:\\Program Files\\Python312\\Lib\\site-packages\\win32\\lib', 'c:\\Program Files\\Python312\\Lib\\site-packages\\Pythonwin']
Backtesting was not in sys path
path after ['c:\\Program Files\\Python312\\python312.zip', 'c:\\Program Files\\Python312\\DLLs', 'c:\\Program Files\\Python312\\Lib', 'c:\\Program Files\\Python312', '', 'C:\\Users\\vinayak\\AppData\\Roaming\\Python\\Python312\\site-packages', 'c:\\Program Files\\Python312\\Lib\\site-packages', 'c:\\Program Files\\Python312\\Lib\\site-packages\\win32', 'c:\\Program Files\\Python312\\Lib\\site-packages\\win32\\lib', 'c:\\Program Files\\Python312\\Lib\\site-packages\\Pythonwin', 'C:

In [3]:
# from scipy.optimize import minimize
import TradeAndLogics as TL
import Plot
import Data as data
import Data_Processing as dp
from enums import *
from Utility import *

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
pwd()

In [ ]:
original_stdout = const(sys.stdout)
original_stdout.value

TRADING/ STRATEGY PARAMETERS

In [7]:
index_symbol = 'BANKNIFTY'
start_date = '2023-07-01'
end_date = '2023-12-01'
expiry_type_near = 'I'
expiry_type_next = 'II'
risk_free_rate = 0.1 # (10% interest rate)
timeframe = 15 # mins
look_back_window = 5*25*4
# Trade/ Strategy Parameters
index_delta_threshold_per_lot = 5
component_delta_threshold_per_lot = 1
notional_vega = 10000 # rs
profit_target = 50000
stop_loss = 50000
zscore_threshold_long = 1.5
zscore_threshold_short = -1
epsilon = 0.2

In [8]:
strategy_desc = f"Dispersion_AllComponents_IV_Normalization_{index_symbol}"

In [9]:
def round(x, dig=2):
    return np.round(x, dig)

In [10]:
def insert_index_component(constituents, constituent_symbol, weight, lot_size):
    dict = {
        'weight': weight,
        'lot_size': lot_size
    }
    constituents[constituent_symbol] = dict

In [11]:
constituents_weight_lots = {}

In [12]:
index_symbol = 'BANKNIFTY'
index_lot_size = 15

In [13]:
insert_index_component(constituents_weight_lots, "PNB", 0.91, 8000)
insert_index_component(constituents_weight_lots, "ICICIBANK", 23.03, 700)
insert_index_component(constituents_weight_lots, "AUBANK", 2.69, 1000)
insert_index_component(constituents_weight_lots, "BANKBARODA", 1.84, 2925)
insert_index_component(constituents_weight_lots, "FEDERALBNK", 1.68, 5000)
insert_index_component(constituents_weight_lots, "IDFCFIRSTB", 1.08, 7500)
insert_index_component(constituents_weight_lots, "SBIN", 11.27, 750)
insert_index_component(constituents_weight_lots, "INDUSINDBK", 5.58, 500)
insert_index_component(constituents_weight_lots, "HDFCBANK", 27.04, 550)
insert_index_component(constituents_weight_lots, "KOTAKBANK", 11.72, 400)
insert_index_component(constituents_weight_lots, "BANDHANBNK", 1.98, 2800)
insert_index_component(constituents_weight_lots, "AXISBANK", 11.18, 625)

In [ ]:
constituents_weight_lots

In [15]:
def normalize_constituents_weights(constituents_weight_lots):
    weight_sum = 0
    for values in constituents_weight_lots.values():
        weight_sum += values['weight']
    if weight_sum != 100:
        print(f"Sum of weights = {weight_sum} is not 100. Normalizing weights...")
        total_weight = sum([values['weight'] for values in constituents_weight_lots.values()])
        for values in constituents_weight_lots.values():
            values['weight'] = (values['weight'] / total_weight) * 100
    
        normalized_sum = sum([values['weight'] for values in constituents_weight_lots.values()])
        print("Normalized sum:", normalized_sum)
        print(constituents_weight_lots)
    else:
        print(f"provided weights add up to perfect {weight_sum}")
    return (constituents_weight_lots)

In [ ]:
normalize_constituents_weights(constituents_weight_lots)

NEAR MONTH (I) DATA FETCHING AND PROCESSING

In [17]:
constituents_weights_lots_data_near = {}

In [18]:
logs_fetching = open(f'LOGS_Fetching_Near_Month_Data_for_{strategy_desc}.txt', 'w')
sys.stdout = logs_fetching
for constituent_symbol, constituent in constituents_weight_lots.items():
    df_futures_near = data.get_futures_data_with_timestamps_between(constituent_symbol, expiry_type_near, start_date, end_date, timeframe)
    df_options_near = data.get_options_data_with_timestamps_between(constituent_symbol, expiry_type_near, start_date, end_date, timeframe)
    # constituents_weights_lots_data_near[constituent_symbol] = constituents_weight_lots[constituent_symbol]
    for key, value in constituents_weight_lots.items():
        constituents_weights_lots_data_near[key] = value
    constituents_weights_lots_data_near[constituent_symbol]['df_options'] = df_options_near
    constituents_weights_lots_data_near[constituent_symbol]['df_futures'] = df_futures_near
sys.stdout = original_stdout.value
logs_fetching.close()

In [19]:
logs_fetching = open(f'LOGS_Fetching_Near_Month_Data_for_{strategy_desc}.txt', 'a')
sys.stdout = logs_fetching
df_futures_index_near = data.get_futures_data_with_timestamps_between(index_symbol, expiry_type_near, start_date, end_date, timeframe)
df_options_index_near = data.get_options_data_with_timestamps_between(index_symbol, expiry_type_near, start_date, end_date, timeframe)
sys.stdout = original_stdout.value
logs_fetching.close()

In [196]:
constituents_near = {}

In [197]:
index_near = dp.ticker(index_symbol, df_futures_index_near, df_options_index_near, True, constituents_near, False, None, index_lot_size, True, timeframe, look_back_window)
for constituent_symbol, constituent in constituents_weights_lots_data_near.items():
    constituents_near[constituent_symbol] = dp.ticker(constituent_symbol, constituent['df_futures'], constituent['df_options'], False, None, True, constituent['weight']/100, constituent['lot_size'], True, timeframe, look_back_window, False)

In [189]:
ohlc = OHLC.close

In [190]:
index_near.set_ohlc(ohlc)
for _, constituent in index_near.components.items():
    constituent.set_ohlc(ohlc)

NEXT MONTH (II) DATA FETCHING AND PROCESSING

In [24]:
constituents_weights_lots_data_next = {}

In [25]:
logs_fetching = open(f'LOGS_Fetching_Next_Month_Data_for_{strategy_desc}.txt', 'w')
sys.stdout = logs_fetching
for constituent_symbol, constituent in constituents_weight_lots.items():
    df_futures_next = data.get_futures_data_with_timestamps_between(constituent_symbol, expiry_type_next, start_date, end_date, timeframe)
    df_options_next = data.get_options_data_with_timestamps_between(constituent_symbol, expiry_type_next, start_date, end_date, timeframe)
    # constituents_weights_lots_data_near[constituent_symbol] = constituents_weight_lots[constituent_symbol]
    for key, value in constituents_weight_lots.items():
        constituents_weights_lots_data_next[key] = value
    constituents_weights_lots_data_next[constituent_symbol]['df_options'] = df_options_next
    constituents_weights_lots_data_next[constituent_symbol]['df_futures'] = df_futures_next
sys.stdout = original_stdout.value
logs_fetching.close()

In [26]:
logs_fetching = open(f'LOGS_Fetching_Next_Month_Data_for_{strategy_desc}.txt', 'a')
sys.stdout = logs_fetching
df_futures_index_next = data.get_futures_data_with_timestamps_between(index_symbol, expiry_type_next, start_date, end_date, timeframe)
df_options_index_next = data.get_options_data_with_timestamps_between(index_symbol, expiry_type_next, start_date, end_date, timeframe)
sys.stdout = original_stdout.value
logs_fetching.close()

In [191]:
constituents_next = {}

In [192]:
index_next = dp.ticker(index_symbol, df_futures_index_next, df_options_index_next, True, constituents_next, False, None, index_lot_size, False, timeframe, look_back_window)
for constituent_symbol, constituent in constituents_weights_lots_data_next.items():
    constituents_next[constituent_symbol] = dp.ticker(constituent_symbol, constituent['df_futures'], constituent['df_options'], False, None, True, constituent['weight']/100, constituent['lot_size'], False, timeframe, look_back_window, False)

In [193]:
ohlc = OHLC.close

In [194]:
index_next.set_ohlc(ohlc)
for _, constituent in index_next.components.items():
    constituent.set_ohlc(ohlc)

BEGINS

In [32]:
def get_index_and_components_iv(index):
    index.generate_iv_data()
    for _, component in index.components.items():
        component.generate_iv_data()

In [181]:
logs_iv_near = open(f'LOGS_IV_Generation_for_Near_Month_{strategy_desc}.txt', 'w')
sys.stdout = logs_iv_near
get_index_and_components_iv(index_near)
sys.stdout = original_stdout.value
logs_iv_near.close()

In [182]:
logs_iv_next = open(f'LOGS_IV_Generation_for_Next_Month_{strategy_desc}.txt', 'w')
sys.stdout = logs_iv_next
get_index_and_components_iv(index_next)
sys.stdout = original_stdout.value
logs_iv_next.close()

NORMALIZE HORIZONTALLY OVER HERE

In [35]:
def get_normallized_iv(timestamp, ticker_near, ticker_next):
    print(f"> {ticker_near.symbol}")
    iv_near = ticker_near.df_futures.loc[timestamp, 'iv']
    iv_next = ticker_next.df_futures.loc[timestamp, 'iv']
    print(f">>  IV Near: {iv_near} || IV Next: {iv_next}")
    expiry_near = pd.to_datetime(ticker_near.df_futures.loc[timestamp, 'expiry'])
    expiry_next = pd.to_datetime(ticker_next.df_futures.loc[timestamp, 'expiry'])
    print(f">>  Expiry Near: {expiry_near} || Expiry Next: {expiry_next}")
    time_to_expiry_near = (expiry_near - timestamp).days
    time_to_expiry_next = (expiry_next - timestamp).days
    print(f">>  Time to Expiry Near: {time_to_expiry_near}days || Time to Expiry Next: {time_to_expiry_next}days")
    weight1 = (time_to_expiry_next - 30)/(time_to_expiry_next - time_to_expiry_near) if iv_next != 0 else 1
    weight2 = 1 - weight1
    print(f">>  Weight1 Near: {weight1} || Weight2 Next: {weight2}")
    normalized_iv = weight1 * iv_near + weight2 * iv_next
    print(f">>  Normallized IV = {normalized_iv}")
    return normalized_iv

In [36]:
def get_index_with_normalized_iv(index_near, index_next):
    logs_normalization_iv = open(f'LOGS_IV_Normalization_for_{strategy_desc}.txt', 'w')
    sys.stdout = logs_normalization_iv
    index_fresh = copy.deepcopy(index_near)
    print("Normallizing IV horizontally using Near and Next month options")
    for timestamp in index_near.df_futures.index:
        print(timestamp)
        index_fresh.df_futures.loc[timestamp, 'iv'] = get_normallized_iv(timestamp, index_near, index_next)
        for component_near, component_next in zip(index_near.components.values(), index_next.components.values()):
            index_fresh.df_futures.loc[timestamp, 'iv'] = get_normallized_iv(timestamp, component_near, component_next)
        print('-------------------------------------------------------------')
    sys.stdout = original_stdout.value
    logs_normalization_iv.close()
    return index_fresh

In [ ]:
Index = get_index_with_normalized_iv(index_near, index_next)

BACK TO NORMAL

In [ ]:
Index.generate_ic_data()

In [ ]:
index = copy.deepcopy(Index)

In [39]:
index.df_futures['zscore'] = (index.df_futures['ic'] - index.df_futures['ic'].rolling(window=index.look_back_window).mean())/index.df_futures['ic'].rolling(window=index.look_back_window).std()

In [ ]:
print(index.df_futures['zscore'])

In [ ]:
fig_ic = Plot.plot_df(index.df_futures, 'ic')
fig_ic.show()
Plot.save_plot(fig_ic, f'PLOT_ic_for_{strategy_desc}.html')

In [ ]:
fig_zscore = Plot.plot_df(index.df_futures, 'zscore')
fig_zscore = Plot.draw_horizontal_line(fig_zscore, -2, 0, len(index.df_futures), 'Red')
fig_zscore = Plot.draw_horizontal_line(fig_zscore, zscore_threshold_short, 0, len(index.df_futures), 'Blue')
fig_zscore = Plot.draw_horizontal_line(fig_zscore, zscore_threshold_long, 0, len(index.df_futures), 'Blue')
fig_zscore = Plot.draw_horizontal_line(fig_zscore, 2, 0, len(index.df_futures), 'Red')
fig_zscore.show()
Plot.save_plot(fig_zscore, f'PLOT_zscore_for_{strategy_desc}.html')

In [ ]:
x = index.df_futures['zscore']
count = 0
flag = False
for z in x:
    if z > zscore_threshold_long and flag == False:
        flag = True
        count+=1
    if z < zscore_threshold_short and flag == True:
        flag = False
        count += 1
print("theoretically number of trades just with zscore signals", count)

In [44]:
positions = {}

In [156]:
def get_straddle_vega(ticker, timestamp):
    vega = 0
    for option_type in Option:
        strike, _ = ticker.find_moneyness_strike(timestamp, 0, option_type)
        greeks = ticker.greeks(timestamp, option_type, strike)
        if greeks is None:
            print(f"get_straddle_vega: greeks is None || Symbol: {ticker.symbol} | OHLC: {ticker.ohlc.name} | Spot: {ticker.get_futures_price(timestamp)} | Strike: {strike} | Opt Price: {ticker.get_opts(option_type).loc[timestamp, strike]} | T(days): {pd.to_datetime(ticker.get_opts(option_type).loc[timestamp, 'expiry']).date() - pd.to_datetime(timestamp).date()} | Option: {option_type.name}")
            return None
        vega += greeks['vega']
    return vega

In [157]:
def get_lots_for_neutrality(vegas, weights, notional_vega, logging_information):
    # lots_index = notional_vega/ vegas[0]
    # lots_components = []
    # for vega_component in vegas[1:]:
    #     lots_components.append(notional_vega/vega_component)
    lots_for_neutrality = []
    for symbol, _ in vegas.items():
        lots = notional_vega * weights[symbol] / vegas[symbol]
        lots_for_neutrality.append(lots)
        logging_information[symbol]['Lots to take position'] = int(lots)
    return np.round(lots_for_neutrality[0]).astype(int), np.round(lots_for_neutrality[1:]).astype(int)

In [158]:
def entry_signal(timestamp):
    zscore = index.df_futures.loc[timestamp, 'zscore']
    return zscore >= zscore_threshold_long


def exit_signal(timestamp):
    zscore = index.df_futures.loc[timestamp, 'zscore'] 
    return zscore <= zscore_threshold_short

def get_lots_for_dispersion(dispersion_position_of_index, index, timestamp, strike_notional = False, vega_neutral = False, theta_neutral = False, notional_vega = 100000):
    # If you want to do vega neutral entry
    if vega_neutral: 
        try:
            unweighted_vegas, weights, logging_information = {}, {}, {}
            vega_index_atm_straddle = get_straddle_vega(index, timestamp)
            if vega_index_atm_straddle is None:
                return None, None
            unweighted_vegas[index.symbol] = vega_index_atm_straddle * index.lot_size
            weights[index.symbol] = 1
            logging_information[index.symbol] = {
                'Lot Size': index.lot_size,
                'Weight': index.weight,
                'Vega per Straddle': vega_index_atm_straddle,
                'Vega per Lot Straddle': vega_index_atm_straddle * index.lot_size,
                'Target Notional Vega': notional_vega
            }
            for _, component in index.components.items():
                vega_component_atm_straddle = get_straddle_vega(component, timestamp)
                if vega_component_atm_straddle is None:
                    return None, None
                unweighted_vegas[component.symbol] = vega_component_atm_straddle * component.lot_size
                weights[component.symbol] = component.weight
                logging_information[component.symbol] = {
                    'Lot Size': component.lot_size,
                    'Weight': component.weight,
                    'Vega per Straddle': vega_component_atm_straddle,
                    'Vega per Lot Straddle': vega_component_atm_straddle * component.lot_size,
                    'Target Notional Vega': notional_vega * component.weight
                }
            lots_index, lots_components = get_lots_for_neutrality(unweighted_vegas, weights, notional_vega, logging_information)
            for symbol, symbol_information in logging_information.items():
                # information_string = " || ".join([f"{info_key} = {info_value}" for info_key, info_value in value.items()])
            # output_string = f"{key} || {information_string}"
                print(f"{symbol} || Lots: {symbol_information['Lots to take position']}")
                for info_key, info_value in symbol_information.items():
                    if(info_key == "Lots to take position"):
                        continue
                    print(f"  {info_key}: {info_value}")
            return lots_index, lots_components
        except Exception as e:
            print(f"get_lots_for_dispersion() error: {e}")
    # If you want to do a delta neutral entry
    if strike_notional:
        pass
    # If you want to do a theta/ correlation neutral entry
    if theta_neutral:
        pass

In [159]:
def get_net_delta(ticker, timestamp):
    net_delta = 0
    for _, token in ticker.tokens.items():
        net_delta += token.stats.loc[timestamp, 'net_delta']
    return net_delta

In [160]:
def take_dispersion_position(time_data, remarks, ticker, position, lots):
    timestamp, time_ix = time_data
    legs = []
    for option_type in Option:
        atm_strike, _ = ticker.find_moneyness_strike(timestamp, 0, option_type)
        key = f'{ticker.symbol}_{option_type.name}_{atm_strike}' 
        if key not in positions:
            positions[key] = TL.Token(ticker, option_type, atm_strike, FNO.OPTIONS)
        token = positions[key]
        token.add_position(timestamp, lots, position)
        token.update_df(time_ix, timestamp)
        ticker.update_token(key, token)
        leg = {
            "position" : position, 
            "opt_type" : option_type, 
            "strike": atm_strike, 
            "lots": lots
        }
        legs.append(leg)
    ticker.take_position(timestamp, remarks, *legs)


In [161]:
def check_existing_position(ticker):
    pos = False
    for _, token in ticker.tokens.items():
        pos = pos | (token.position.value != 0) 
    return pos

In [162]:
def hedge_delta(time_ix, timestamp, remarks, ticker, index_delta_threshold_per_lot = 5, component_delta_threshold_per_lot = 1, logging = True):
    net_delta = get_net_delta(ticker, timestamp)
    delta_threshold_per_lot = index_delta_threshold_per_lot if ticker.is_index else component_delta_threshold_per_lot 
    if abs(net_delta) >= delta_threshold_per_lot * ticker.lot_size:
        qty_futures = (-net_delta)/ticker.lot_size
        position = LongShort(np.sign(qty_futures))
        lots_futures = int(abs(qty_futures))
        if lots_futures != 0:
            ticker.hedge_futures_trade(lots_futures, position, timestamp)
            key = f'{ticker.symbol}_Futures'
            if key not in positions:
                positions[key] = TL.Token(ticker, None, None, FNO.FUTURES)
            token = positions[key]
            token.add_position(timestamp, lots_futures, position)
            token.update_df(time_ix, timestamp, logging)
            ticker.update_token(key, token) #python obj ref prop, check
            token.ticker = ticker
            positions[key] = token #python obj ref prop, check
            leg = {
                "position" : position, 
                "opt_type" : FNO.FUTURES, 
                "strike": None,
                "lots": lots_futures
            }
            ticker.take_position(timestamp, remarks, leg)
    
            print(f">> HEDGED DELTA {ticker.symbol}: Added {lots_futures * position.value * ticker.lot_size} notional delta using {lots_futures} lots of future. new delta = {round(net_delta) + lots_futures * position.value * ticker.lot_size} || threshold per option = {delta_threshold_per_lot}")
        else:
            print(f">> Hedging condition threshold per option = {delta_threshold_per_lot} for {ticker.symbol} met but required lots_futures were 0 so did not hedge")
    else:
        print(f"XXXX DID NOT HEDGE  {ticker.symbol}'s {round(net_delta)} notional delta for {ticker.symbol} at {timestamp}. net_delta/{ticker.symbol}_lot_size_futures = {np.round(abs(net_delta)/ticker.lot_size)} < 1")

In [163]:
def reset_hedging_positions(ticker):
    ticker.hedging = pd.DataFrame(columns = ['date_timestamp', 'lots', 'position', 'futures_price', 'delta_added'])
    ticker.hedging.set_index('date_timestamp', inplace=True)

def reset_trade_positions(ticker):
    ticker.Trades = dp.Trades()
    ticker.tokens = {}

In [164]:
def HandleUpdate(ticker, time_ix, timestamp, remarks, hedge = True, logging_token_update=True, logging_hedging=True, index_delta_threshold_per_lot = 5, component_delta_threshold_per_lot = 1):
    for key, token in ticker.tokens.items():
        token.update_df(time_ix, timestamp, logging_token_update)
        positions[key] = token #python obj ref prop, check
        if not logging_hedging:
            continue
        if token.fno == FNO.OPTIONS:
            print(f"{token.ticker.symbol, token.opt_type.name, float(token.strike/100)} | Delta per lot = {np.round(token.stats.loc[timestamp, 'net_delta'], 2)} (position accounted) | Notional Delta = {np.round(token.stats.loc[timestamp, 'net_delta'], 2) * ticker.lot_size} | Lot size: {ticker.lot_size} | Lots in position: {token.lots} | position = {token.position.name}")
        else:
            print(f"{token.ticker.symbol, token.fno.name} | Delta per lot = {np.round(token.stats.loc[timestamp, 'net_delta'], 2)} | Notional Delta = {np.round(token.stats.loc[timestamp, 'net_delta'], 2) * ticker.lot_size} | Lot size: {ticker.lot_size} | Lots in position: {token.lots} | position = {token.position.name}")
    if hedge:
        hedge_delta(time_ix, timestamp, remarks, ticker, index_delta_threshold_per_lot, component_delta_threshold_per_lot)
    for key, token in ticker.tokens.items():
        token.ticker = ticker
        positions[key] = token #python obj ref prop, check

In [165]:
reload_modules()

In [166]:
class DispersionPTSL(TL.PTSLHandling):
    def __init__(self, profit_target, stop_loss, *tickers):
        super().__init__(profit_target, stop_loss, *tickers)

    def is_valid(self, timestamp):
        return not self.active_ptsl   
    
    def update_validity(self, timestamp):
        index = self.tickers[0]
        if self.triggered_at is None:
            return
        for z in index.df_futures['zscore'].loc[self.triggered_at : timestamp]:
            if abs(z) <= epsilon:
                self.triggered_at = None
                self.active_ptsl = False  
                print(f"PTSL reverted back to mean (<= {epsilon}): Future Signals can now be used for trading")

In [169]:
def squareoff_ticker(time_ix, timestamp, remarks, ticker, logging):
    legs_trade, leg_hedge = [], []
    if(not check_existing_position(ticker)):
        print(f"{ticker.symbol} does not hold any position")
        return
    print(f"Squaring off {ticker.symbol}")
    for key, token in ticker.tokens.items():
        if token.position == LongShort.Neutral:
            continue
        position_to_take = LongShort(-token.position.value) 
        if token.fno == FNO.OPTIONS:
            leg = {
            "position" : position_to_take, 
            "opt_type" : token.opt_type, 
            "strike": token.strike,
            "lots": token.lots
            }
            legs_trade.append(leg)
        else:
            leg = {
            "position" : position_to_take, 
            "opt_type" : FNO.FUTURES, 
            "strike": token.strike,
            "lots": token.lots
            }
            leg_hedge.append(leg)
        token.add_position(timestamp, token.lots, position_to_take)
        token.update_df(time_ix, timestamp, logging)
        positions[key] = token #python obj ref prop, check
        ticker.tokens[key] = token #python obj ref prop, check
    if len(legs_trade):
        ticker.take_position(timestamp, remarks, *legs_trade)
    else:    
        print(f"Did not find any trading legs for {ticker.symbol}")
    if len(leg_hedge):
        ticker.take_position(timestamp, remarks, *leg_hedge)
    else:
        print(f"Did not find any hedging legs for {ticker.symbol}")
        
    allSecurities = leg_hedge + legs_trade
    if len(allSecurities):
        secDesc = [f'{ticker.get_expiry(timestamp).strftime('%d%b')}{int(leg['strike']) if leg['strike'] is not None else ""}{leg['opt_type'].name}' for leg in allSecurities]
        print(f"Squared off {", ".join(secDesc)}")

    # Every Token in a Ticker should have the updated Ticker in them
    for key, token in ticker.tokens.items():
        token.ticker = ticker #python obj ref prop, check
        positions[key] = token #python obj ref prop, check

In [128]:
def squareoff(time_ix, timestamp, remarks, logging = True):
    print("************  SQUARE OFF BEGINS  *********** ")
    # for key, token in positions.items():
    #     token.add_position(token.lots, LongShort(-token.position.value))
    # print("******  positions added in GLOBAL TOKENS. Now updating the key-information(running_pnl, lots, !(delta, vega, iv)) in them  ******")
    # for key, token in positions.items():
    #     token.update_df(time_ix, timestamp, logging)
    #     positions[key] = token
    # print("******  Information updated globally. TICKER WISE UPDATION BEGINS  ******")
    squareoff_ticker(time_ix, timestamp, remarks, index, logging)
    for component in index.components.values():
        squareoff_ticker(time_ix, timestamp, remarks, component, logging)
    print("* SQUARE OFF TRADES SAVED, TOKENS IN TICKERS UPDATED")
    print("************  SQUARE OFF COMPLETE  *********** ")


In [129]:
def isTodayAnyExpiry(timestamp, index):
    is_expiry = False
    is_expiry = is_expiry or (pd.to_datetime(timestamp).date() == pd.to_datetime(index.get_opts(Option.Call).loc[timestamp, 'expiry']).date())
    for component in index.components.values():
        is_expiry = is_expiry or (pd.to_datetime(timestamp).date() == pd.to_datetime(component.get_opts(Option.Call).loc[timestamp, 'expiry']).date())
    return is_expiry

In [130]:
def UpdateDispersionTickers(time_ix, timestamp, remarks, hedge, logging_token_update, logging_hedging):
    HandleUpdate(index, time_ix, timestamp, remarks, 
                 hedge, logging_token_update, logging_hedging, 
                 index_delta_threshold_per_lot=index_delta_threshold_per_lot, 
                 component_delta_threshold_per_lot=component_delta_threshold_per_lot)
    for component in index.components.values():
        HandleUpdate(component, time_ix, timestamp, remarks, 
                     hedge, logging_token_update, logging_hedging, 
                     index_delta_threshold_per_lot=index_delta_threshold_per_lot, 
                     component_delta_threshold_per_lot=component_delta_threshold_per_lot)

In [144]:
reload_modules()

In [ ]:
positions = {}
reset_hedging_positions(index)
reset_trade_positions(index)
for component in index.components.values():
    reset_hedging_positions(component)
    reset_trade_positions(component)


PTSL = DispersionPTSL(profit_target, stop_loss, index, *index.components.values())

file = open(f'LOGS_Trading_Logic_for_{strategy_desc}.txt', 'w')
sys.stdout = file

for time_ix, timestamp in enumerate(index.df_futures.index):
    print()
    print(f"Timestamp: {timestamp}")
    print(f"Z-Score = {index.df_futures.loc[timestamp, 'zscore']}")
    #######################################################################################################################################################
    PTSL.update_validity(timestamp)
    if check_existing_position(index):
        
        print("Existing Position Check: TRUE")
        
        # EXPIRY DAY MANUAL SQUAREOFF
        if isTodayAnyExpiry(timestamp, index):
            print("************  EXPIRY DAY, MANUAL SQUARE OFF ************  ")
            squareoff(time_ix, timestamp, 'Expiry Day, Manual Square Off', False)
            UpdateDispersionTickers(time_ix, timestamp, 'NoHedging', False, False, False)
            continue
            
            
        # PROFIT TARGET AND STOP LOSS SQUAREOFF
        ptsl_result = PTSL.status(timestamp) 
        if ptsl_result != "good_to_go":
            print("************  PTSL Trigger hit *********** ")
            print(f"{PTSL.nature} Square off at NetPnl of {PTSL.pnl_last_trade}")
            squareoff(time_ix, timestamp, f'{ptsl_result} Square Off', True)
            UpdateDispersionTickers(time_ix, timestamp, 'NoHedging', False, True, False)
            continue
            
            
        # REVERSE SIGNAL SQUAREOFF
        squareoff_signal = exit_signal(timestamp) if flag else entry_signal(timestamp)
        if squareoff_signal:
            print("Entered squareoff signal block")
            print("************  SquareOff Reverse zscore Signal hit. ************ ")
            squareoff(time_ix, timestamp, 'ZScore Reversal Square Off (Entry in next timestamp/ signal)')
            UpdateDispersionTickers(time_ix, timestamp, 'NoHedging', False, True, False)
            continue
            
        # IF NO NEED TO SQUAREOFF, HEDGE IF NEEDED
        UpdateDispersionTickers(time_ix, timestamp, 'Hedge', True, True, True)
        print("================")
        continue
    #######################################################################################################################################################
    
    
    #######################################################################################################################################################
    if entry_signal(timestamp):
        print("************  LONG DISPERSION SIGNAL ************  ")
        if isTodayAnyExpiry(timestamp, index):
            print("Expiry Day today. Trading on Expiry Day is not Allowed")
            UpdateDispersionTickers(time_ix, timestamp, "NoHedging", False, False, False)
            continue

        if not PTSL.is_valid(timestamp):
            print(f"{PTSL.nature} was Triggered at {PTSL.triggered_at.strftime('%H:%M on %d/%b/%Y')} and Z-Score has not yet reverted back to mean (abs(z) is not yet <= {epsilon})")
            UpdateDispersionTickers(time_ix, timestamp, "NoHedging", False, True, False)
            continue
        
        try:
            lots_index, lots_components = get_lots_for_dispersion(LongShort.Short, index, timestamp, False, True, False, notional_vega)
            # print(f"Index Lots: {lots_index} || Components Lots: {lots_index, lots_components}")

            if lots_index == 0:
                print("Cant take Dispersion Position, lots index for target notional vega can't be 0")
                UpdateDispersionTickers(time_ix, timestamp, "NoHedging", False, True, False)
                continue

            take_dispersion_position((timestamp, time_ix), 'Long Dispersion Signal (zscore high, short index, long components)', index, LongShort.Short, lots_index)
            for (component, lots_component) in zip(index.components.values(), lots_components):
                take_dispersion_position((timestamp, time_ix), 'Long Dispersion Signal (zscore high, short index, long components)', component, LongShort.Long, lots_component)
            PTSL.fresh_trade(timestamp)
            print(f"Long dispersion (Short Index Long Components) trade executed")
        except Exception as e:
            print(f"Couldn't Enter a Long Dispersion Position: {e}")  
    #######################################################################################################################################################


    #######################################################################################################################################################
    elif exit_signal(timestamp):
        print("************  SHORT DISPERSION SIGNAL ************  ")
        if isTodayAnyExpiry(timestamp, index):
            print("Expiry Day today. Trading on Expiry Day is not Allowed")
            UpdateDispersionTickers(time_ix, timestamp, "NoHedging", False, False, False)
            continue

        if not PTSL.is_valid(timestamp):
            print(f"{PTSL.nature} was Triggered at {PTSL.triggered_at.strftime('%H:%M on %d/%b/%Y')} and Z-Score has not yet reverted back to mean (abs(z) is not yet <= {epsilon})")
            UpdateDispersionTickers(time_ix, timestamp, "NoHedging", False, True, False)
            continue

        try:
            lots_index, lots_components = get_lots_for_dispersion(LongShort.Long, index, timestamp, False, True, False, notional_vega)
            # print(f"lots_index, lots_components: {lots_index, lots_components}")

            if lots_index == 0:
                print("Cant take Dispersion Position, lots index for target notional vega can't be 0")
                UpdateDispersionTickers(time_ix, timestamp, "NoHedging", False, True, False)
                continue
            
            take_dispersion_position((timestamp, time_ix), 'Short Dispersion Signal (zscore low, long index, short components)', index, LongShort.Long, lots_index)
            for (component, lots_component) in zip(index.components.values(), lots_components):
                take_dispersion_position((timestamp, time_ix), 'Short Dispersion Signal (zscore low, long index, short components)', component, LongShort.Short, lots_component)
            PTSL.fresh_trade(timestamp)
            print(f"Short dispersion (Long Index Short Components) trade")
        except Exception as e:
            print(f"Couldn't Enter a Short Dispersion Position: {e}")  
    #######################################################################################################################################################

UpdateDispersionTickers(time_ix, timestamp, "NoHedging", False, True, False)
    

file.close()
sys.stdout = original_stdout.value

In [94]:
data = {
    'running_pnl': [],
    'net_delta': [],
    'net_vega': [],
    'net_lots': [],
    'active_position': []
}

for timestamp in index.df_futures.index:
    pnl = 0
    delta = 0
    vega = 0
    lots = 0

    for _, token in positions.items():
        stats_at_timestamp = token.stats.loc[timestamp]
        pnl += stats_at_timestamp['running_pnl']
        delta += stats_at_timestamp['net_delta']
        vega += stats_at_timestamp['net_vega']
        lots += stats_at_timestamp['net_lots']

    active_position = int(abs(lots) > 0)
    
    data['running_pnl'].append(pnl) 
    data['net_delta'].append(delta)
    data['net_vega'].append(vega)
    data['net_lots'].append(lots)
    data['active_position'].append(active_position)
result_df = pd.DataFrame(data, index=index.df_futures.index)

In [95]:
result_df['active_position'] *= 500

In [96]:
fig_pnl = Plot.plot_df(result_df, 'running_pnl')
Plot.save_plot(fig_pnl, f'PLOT_PNL_for_{strategy_desc}.html')

In [97]:
fig_greeks = Plot.plot_df(result_df, 'net_delta', 'net_vega')
Plot.save_plot(fig_greeks, f'PLOT_GREEKS_for_{strategy_desc}.html')

In [98]:
fig_signals = Plot.plot_df(index.df_futures, 'zscore', 'ic')
fig_signals = Plot.draw_horizontal_line(fig_signals, zscore_threshold_short, index.df_futures['ix'].iloc[0], index.df_futures['ix'].iloc[-1], 'white')
fig_signals = Plot.draw_horizontal_line(fig_signals, zscore_threshold_long, index.df_futures['ix'].iloc[0], index.df_futures['ix'].iloc[-1], 'white')
Plot.save_plot(fig_signals, f'PLOT_SIGNALS_for_{strategy_desc}.html')

In [99]:
arr = []
arr += [{**trade, 'symbol': index.symbol} for trade in index.Trades.tradesArr]
for comp in index.components.values():
    arr += [{**trade, 'symbol': comp.symbol} for trade in comp.Trades.tradesArr]

In [ ]:
df_trades = pd.DataFrame(arr)
df_trades = df_trades.sort_values(by='timestamp')
df_trades.reset_index(inplace=True)
df_trades.drop(columns=['index'], inplace=True)

In [74]:
df_trades.to_excel(f"INFO_Trades_for_{strategy_desc}.xlsx", index=False)
Plot.save_excel_trades(f"INFO_Trades_for_{strategy_desc}.xlsx")

In [75]:
def metrics(fund_blocked):
    info_metrics = open(f'INFO_Metrics_for_{strategy_desc}.txt', 'w')
    sys.stdout = info_metrics

    print(f"TimeFrame Length: 6 months")
    print(f"Fundblocked in rs: {fund_blocked/100}")
    result_df.index = pd.to_datetime(result_df.index)
    result_df['date'] = result_df.index.date  # Create a new column for the date
    daily_profit = result_df.groupby('date')['running_pnl'].agg(lambda x: x.iloc[-1] - x.iloc[0])
    daily_profit.index = pd.to_datetime(result_df['date'].unique())
    
    ret = daily_profit.sum()/fund_blocked * 100
    print(f"Return: {round(ret)}%")
    
    sharpe_value = (daily_profit.mean() - risk_free_rate*fund_blocked/365)/daily_profit.std()
    print(f"Sharpe: {round(sharpe_value)}")
    
    peaks = result_df['running_pnl'].cummax()
    drawdowns = (result_df['running_pnl'] - peaks)/(peaks)
    max_drawdown = drawdowns.min()
    print(f"Drawdown: {round(max_drawdown)}%")
    monthly_data = daily_profit.resample('ME').sum()
    monthly_profit = monthly_data[monthly_data > 0].sum()
    monthly_loss = abs(monthly_data[monthly_data < 0].sum())
    if monthly_loss != 0:
        profit_loss_ratio = monthly_profit / monthly_loss
    else:
        profit_loss_ratio = float('inf')  # Avoid division by zero
    print("Monthly Profit/Loss Ratio:", round(profit_loss_ratio))
    
    windays = (result_df['running_pnl'] > 0).sum()
    lossdays = (result_df['running_pnl'] < 0).sum()
    if (windays + lossdays) > 0:
        win_ratio = windays / (windays + lossdays)
    else:
        win_ratio = 0
    print("Win Ratio:", round(win_ratio))
    
    streak = (result_df['running_pnl'] > 0).astype(int)  # 1 for winning days, 0 otherwise
    losing_streak = (result_df['running_pnl'] < 0).astype(int)  # 1 for losing days, 0 otherwise
    winning_streaks = streak.groupby((streak != streak.shift()).cumsum()).cumsum()
    losing_streaks = losing_streak.groupby((losing_streak != losing_streak.shift()).cumsum()).cumsum()
    print(f"Longest Winning Streak: {winning_streaks.max()}, Longest Losing Streak: {losing_streaks.max()}")

    sys.stdout = original_stdout.value
    info_metrics.close()

In [ ]:
metrics(100000)